In [ ]:
# Import libraries for data processing and association rule mining.
from mlxtend.frequent_patterns import association_rules, apriori, fpgrowth  # For implementing association rule mining algorithms.
from mlxtend.preprocessing import TransactionEncoder  # For converting transaction lists into a format suitable for analysis.
import pandas as pd  # For data manipulation and analysis using DataFrames.
import itertools  # For generating combinations and permutations of items.
import time  # For measuring the execution time of algorithms.
import warnings # For fixing the deprecation warnings.

def dataset_loading(file_path):  # Function for loading the dataset from a CSV file and process the transactions.
    try:
        record = pd.read_csv(file_path)  # Reading the CSV file using read_csv() method of pandas library.
        transactions = []
        for t in record['Transaction Details']:  # 'Transaction Details' is the column containing all transactions in the CSV file.
            items = [item.strip() for item in t.split(',')]  # Split each transaction into individual items, stripping spaces.
            transactions.append(items)  # Append the processed transaction (list of items) to the transactions list.
        return transactions
    except Exception as e:
        print(f"\n* Error loading dataset: {e}")# Handling error if any occurs in loading dataset and returns an empty list.
        return []
    except FileNotFoundError:
        print(f"\n* File not found: {file_path}")
        return []
    except pd.errors.ParserError:
        print(f"\n* Error parsing the file: {file_path}")
        return []
    except pd.errors.EmptyDataError:
        print("\n* No data found in the file.")
        return []



def get_all_itemsets(transactions):
    # Step 1: Extract unique items from all transactions using only for loops
    items = set()
    for transaction in transactions:
        for item in transaction:
            items.add(item)  # Add item to the set (to ensure uniqueness)

    # Convert the set to a list for easier manipulation
    items = list(items)

    # Step 2: Generate all itemsets using for loops
    all_itemsets = []

    # Helper function to generate combinations recursively
    def generate_combinations(current_set, idx):
        # If there is any combination, add it to the list
        if current_set:
            all_itemsets.append(tuple(current_set))  # Convert the set to a tuple and add it to the list

        # Generate further combinations
        for i in range(idx, len(items)):
            generate_combinations(current_set + [items[i]], i + 1)

    # Start the recursion with an empty list and index 0
    generate_combinations([], 0)

    return all_itemsets



def support_count(itemset, transactions):  # Function to calculate the support of an itemset in the transactions.
    count = sum(1 for t in transactions if set(itemset).issubset(set(t)))  # Counts the transactions that contain the itemset.
    return count / len(transactions)  # Since, support is: relevant transactions divided by total transactions.



def brute_force_rules(transactions, min_support, min_confidence):  # Function to generate association rules using the brute-force algorithm.
    rules = []
    all_itemsets = get_all_itemsets(transactions)  # Get all possible itemsets using get_all_itemsets(transactions) function.
    if not all_itemsets:
        print("* Can't generate rules as no itemsets are found!")
        return rules  # Returns the function with empty list of rules.
    for itemset in all_itemsets:
        support = support_count(itemset, transactions)  # Calculates support for the itemset.
        if support >= min_support:
            for i in range(1, len(itemset)):
                for conditions in itertools.combinations(itemset, i):  # Use 'conditions' for the antecedents of the rules.
                    results = tuple(set(itemset) - set(conditions))  # Use 'results' for the consquents of the rules.
                    if results:
                        conditions_support = support_count(conditions, transactions)
                        if conditions_support > 0:
                            confidence = support / conditions_support
                            if confidence >= min_confidence:
                                rules.append((conditions, results, support, confidence))  # Append the generated rules in the list.
    return rules # Returns the list of rules generated by Brute Force Algorithm



def display_frequent_itemsets(frequent_itemsets):  # Function to display the frequent itemsets with their support.
    print("\nFrequent Itemsets:\n")
    print("-" * 41)  # Separation line
    print("{:<30} {:<10}".format('|Itemset', ' |Support|'))
    print("-" * 41)  # Separation line
    for index, row in frequent_itemsets.iterrows():
        itemset = tuple(row['itemsets'])  # Get the itemset from the row
        support = row['support']  # Get the support from the row
        print(f"|{str(itemset):<30} | {support * 100:.2f}%|")  # Print itemset and support
    print("-" * 41)  # Separation line


# Collecting all rules into sets for proper comparison
def collect_rules(rules, method):
    rule_set = set()
    for rule in rules:
        conditions, results = rule[0], rule[1]
        rule_set.add(((tuple(sorted(conditions))), (tuple(sorted(results)))))  # Store rules as tuples of antecedents and consequents
    return (rule_set)



# Using dictionary to simulate switch-case for selecting datasets
# Since, the datasets and script is located in same folder, so didn't required to put whole path . If any other dataset is being analyzed then need put whole path.
dataset_files = {
    1: './Datasets_CSV/amazon_csv.csv',
    2: './Datasets_CSV/target_csv.csv',
    3: './Datasets_CSV/walmart_csv.csv',
    4: './Datasets_CSV/aldi_csv.csv',
    5: './Datasets_CSV/costco_csv.csv',
}



def convert_percentage(value):  # Function to convert percentage input of user to decimal for prcoessing.
    return value / 100.0



def main():  # Main function to run the algorithms and provide user interaction.
    warnings.filterwarnings('ignore', category=DeprecationWarning) # Ignoring the depriciation warnings.
    print("\n" + "=" * 100)  # Seperation line
    print("\nWelcome to the Association Rule Mining Program!")

    choice = "y"  # Initialize the choice variable
    while choice.lower() != "n":
        # Prompts for user inputs
        print("\n"+"=" * 100) # Seperation line
        print("=" * 100)  # Seperation line
        print("\nChoose a supermarket for the analysis:")
        print("1. Amazon")
        print("2. Target")
        print("3. Walmart")
        print("4. Aldi")
        print("5. Costco")
        print("6. Provide a custom CSV dataset for the analysis")
        print("7. Exit from program")

        # Input and validation for the choice made by user.
        try:
            dataset_choice = int(input("\nEnter your choice between 1 and 7: "))
            if dataset_choice not in range(1, 8):
                print("\n* Invalid choice. Please choose a number between 1 and 7.\n")
                continue
            elif dataset_choice == 7:  # Option to exit from program.
                break
        except ValueError:
            print("\n* Invalid input. Please enter a number.\n")
            continue



        # Validation and loading the chosen dataset by user.
        if dataset_choice in dataset_files:
            transactions = dataset_loading(dataset_files[dataset_choice])
        elif dataset_choice == 6:
            custom_file = input("Enter the path of your custom CSV file: ")
            transactions = dataset_loading(custom_file)
        else:
            print("\n* Invalid choice. Please try again.")
            continue



        # Input and validation of the support and confidence
        try:
            if transactions == []:
                print("\n* Please try again.")
                continue
            min_support_percent = float(input("Enter the minimum support percentage (1-100): "))
            if not (1 <= min_support_percent <= 100):
                print("\n* Support percentage must be between 1 and 100.\n")
                continue
            min_confidence_percent = float(input("Enter the minimum confidence percentage (1-100): "))
            if not (1 <= min_confidence_percent <= 100):
                print("\n* Confidence percentage must be between 1 and 100.\n")
                continue
        except ValueError:
            print("\n* Invalid input. Please enter numeric values.\n")
            continue



        # Converting percentages to decimals.
        min_support = convert_percentage(min_support_percent)
        min_confidence = convert_percentage(min_confidence_percent)



        # TransactionEncoder is used to convert transaction data (lists of items) into a one-hot encoded format. Each item becomes a column, with 1 indicating
        # its presence in a transaction, and 0 indicating its absence.
        encoder = TransactionEncoder()

        # The fit method learns the unique items from the transaction data and transform method converts the transaction data into a one-hot encoded array.
        onehot = encoder.fit(transactions).transform(transactions)

        # Convert the one-hot encoded array into a DataFrame, where each column corresponds to an item from the original transactions, and the rows represent individual transactions.
        data_frame = pd.DataFrame(onehot, columns=encoder.columns_)



        # Displaying the frequent itemsets with their support.
        print("\n"+"=" * 100)  # Seperation line
        frequent_itemsets = apriori(data_frame, min_support=min_support, use_colnames=True)
        if frequent_itemsets.empty:
            print("\n\n* No frequent itemsets found with the specified support threshold.\n* Please enter proper inputs again")
            continue
        else:
            display_frequent_itemsets(frequent_itemsets)




        # Measuring the execution time and generating rules by each algorithm.

        # For Brute Force Algorithm
        start_time = time.perf_counter()# using perf_counter() method for better accuracy.
        rules_brute_force = brute_force_rules(transactions, min_support, min_confidence)  # Generates rules using Brute Force Algorithm.
        if rules_brute_force == []:
            print("\n\n* No rules found with the specified support and confidence.\n* Please enter proper inputs again")
            continue
        brute_force_time = time.perf_counter() - start_time  # Calculating execution time for Brute Force Algorithm.

        # For Apriori Algorithm
        start_time = time.perf_counter()
        frequent_itemsets_apriori = apriori(data_frame, min_support=min_support, use_colnames=True)
        if frequent_itemsets_apriori.empty:
            print("\n\n* No frequent itemsets found with the specified support threshold.\n* Please enter proper inputs again")
            continue
        rules_apriori = association_rules(frequent_itemsets_apriori, metric="confidence", min_threshold=min_confidence) # Generates rules using Apriori Algorithm.
        apriori_time = time.perf_counter() - start_time  # Calculating execution time for Apriori Algorithm.

        # For FP-Growth Algorithm
        start_time = time.perf_counter()
        frequent_itemsets_fp = fpgrowth(data_frame, min_support=min_support, use_colnames=True)
        if frequent_itemsets_fp.empty:
            print("\n\n* No frequent itemsets found with the specified support threshold.\n* Please enter proper inputs again")
            continue
        rules_fp_growth = association_rules(frequent_itemsets_fp, metric="confidence", min_threshold=min_confidence)  # Generate rules using FP-Growth Algorithm.
        fp_growth_time = time.perf_counter() - start_time  # Calculating execution time for FP-Growth Algorithm.



        # Display rules for each algorithm.

        # For Brute Force Algorithm
        print("=" * 100)  # Seperation line.
        print("\nRules generated using Brute Force Algorithm:")
        for index,rule in enumerate(rules_brute_force):
            conditions, results, support, confidence = rule  # Unpack the rule for printing them.
            conditions = (conditions)
            results = (results)
            print(f"\nRule {index+1}: {conditions} -> {results} has Support: {support * 100:.2f}% and Confidence: {confidence * 100:.2f}%") # Formatted to print percentage of support and confidence rounded upto 2 decimals.

        # For Apriori Algorithm
        print("\n"+"=" * 100)  # Seperation line.
        print("\nRules generated using Apriori Algorithm:")
        for index, row in rules_apriori.iterrows():  # Iterating over DataFrame rows of onehot. Here index is a temporary variable.
            conditions_apriori = (tuple(row['antecedents']))
            results_apriori = (tuple(row['consequents']))
            support = row['support']
            confidence = row['confidence']
            print(f"\nRule {index+1}: {conditions_apriori} -> {results_apriori} has Support: {support * 100:.2f}% and Confidence: {confidence * 100:.2f}%") # Formatted to print percentage of support and confidence rounded upto 2 decimals.

        # For FP-Growth Algorithm
        print("\n"+"=" * 100)  # Seperation line.
        print("\nRules generated using FP-Growth Algorithm:")
        for index, row in rules_fp_growth.iterrows():  # Iterating over DataFrame rows of onehot. Here index is a temporary variable.
            conditions_fp = (tuple(row['antecedents']))
            results_fp = (tuple(row['consequents']))
            support = row['support']
            confidence = row['confidence']
            print(f"\nRule {index+1}: {conditions_fp} -> {results_fp} has Support: {support * 100:.2f}% and Confidence: {confidence * 100:.2f}%") # Formatted to print percentage of support and confidence rounded upto 2 decimals.



        # Comparing the rules generated by algorithms.
        # We used sets for comparison as in this order dosen't matter and also no duplicates are included.
        # Collect rules for brute force, Apriori, and FP-Growth
        brute_force_set = set(collect_rules(rules_brute_force, "Brute Force"))
        apriori_set = set((tuple(sorted(row['antecedents'])), tuple(sorted(row['consequents']))) for index, row in rules_apriori.iterrows())
        fp_growth_set = set((tuple(sorted(sorted(row['antecedents']))), tuple(sorted(row['consequents']))) for index, row in rules_fp_growth.iterrows())

        # Perform set comparison
        print("\n" + "=" * 100)  # Separation line
        print("\nComparison of Rules Generated by the Algorithms:")
        if brute_force_set == apriori_set == fp_growth_set:
            print("\nAll three algorithms generated the same rules.")
        else:
            if brute_force_set == apriori_set:
                print("\nBrute Force and Apriori generated the same rules, but FP-Growth generated different rules.")
            elif brute_force_set == fp_growth_set:
                print("\nBrute Force and FP-Growth generated the same rules, but Apriori generated different rules.")
            elif apriori_set == fp_growth_set:
                print("\nApriori and FP-Growth generated the same rules, but Brute Force generated different rules.")
            else:
                print("\nAll three algorithms generated different rules.")



        # Comparing the execution time and determining the fastest, slowest, and intermediate algorithm.
        times = {
            "Brute Force Algorithm": brute_force_time,
            "Apriori Algorithm": apriori_time,
            "FP-Growth Algorithm": fp_growth_time,
        }

        # Sort the 'times' dictionary (which holds the algorithms and their execution times) by the execution time in ascending order. The lambda function 'x[1]' ensures
        # sorting is based on the values (time), not the keys (algorithm names).
        sorted_algorithms = sorted(times.items(), key=lambda x: x[1])
        fastest = sorted_algorithms[0]
        intermediate = sorted_algorithms[1]
        slowest = sorted_algorithms[2]

        print("\n" + "=" * 100) # Seperation line.
        print("\nComparison of the execution times of algorithm:")
        print(f"\nThe Fastest algorithm is: {fastest[0]} with an execution time of {fastest[1]:.4f} seconds.")
        print(f"\nThe Intermediate algorithm is: {intermediate[0]} with an execution time of {intermediate[1]:.4f} seconds.")
        print(f"\nThe Slowest algorithm is: {slowest[0]} with an execution time of {slowest[1]:.4f} seconds.") # Formatted to print the seconds upto 4 decimals with thier respective algorithms.

        print("\n" + "=" * 100) # Seperation line.
        choice = input("\nDo you want to run another analysis?\nPress Y to continue or N to Exit: ")  # Choice to run the program until user wants.
    print("\n" + "=" * 100) # Seperation line.
    print("\nThank you for using the Association Rule Mining Program! \nHave a Great Day!") #Greeting message.



if __name__ == "__main__":  # Ensuring the main function runs when the script is executed.
    main()





Welcome to the Association Rule Mining Program!


Choose a supermarket for the analysis:
1. Amazon
2. Target
3. Walmart
4. Aldi
5. Costco
6. Provide a custom CSV dataset for the analysis
7. Exit from program

Enter your choice between 1 and 7: 1
Enter the minimum support percentage (1-100): 50
Enter the minimum confidence percentage (1-100): 50


Frequent Itemsets:

-----------------------------------------
|Itemset                        |Support|
-----------------------------------------
|('Books',)                     | 55.00%|
|('Clothes',)                   | 55.00%|
-----------------------------------------


* No rules found with the specified support and confidence.
* Please enter proper inputs again


Choose a supermarket for the analysis:
1. Amazon
2. Target
3. Walmart
4. Aldi
5. Costco
6. Provide a custom CSV dataset for the analysis
7. Exit from program

Enter your choice between 1 and 7: 1
Enter the minimum support percentage (1-100): 20
Enter the minimum confidence perc